In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
import cv2
from skimage.io import imread,imshow
import os
import random
from tqdm import tqdm
from skimage.transform import resize
from sklearn.model_selection import train_test_split

## Performance of vanillla model with image preprocessing

In [127]:
TRAIN_PATH = "dataset/2d_slices/pre_processed_images/"
TRAIN_MASK_PATH = "dataset/2d_slices/masks/"

In [128]:
original_path = "dataset/2d_slices/imgs/"

In [129]:
train_ids = next(os.walk(TRAIN_PATH))[2]
mask_train_ids = next(os.walk(TRAIN_MASK_PATH))[2]
original_ids = next(os.walk(original_path))[2]

In [130]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

In [131]:
X = np.zeros((len(train_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.float16)
Y=np.zeros((len(mask_train_ids),IMG_HEIGHT,IMG_WIDTH,1),dtype = np.float16)
W = np.zeros((len(train_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.float16)

In [18]:
for n,id_ in tqdm(enumerate(train_ids),total=len(train_ids)):
    path=TRAIN_PATH+"/"+id_
    img = np.load(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    X[n]=np.expand_dims(img, axis=2)

100%|██████████████████████████████████████████████████████████████████████████████| 1176/1176 [00:56<00:00, 20.92it/s]


In [19]:
for n,id_ in tqdm(enumerate(mask_train_ids),total=len(mask_train_ids)):
    path=TRAIN_MASK_PATH+"/"+id_
    mask = np.load(path)
    mask = resize(mask,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    Y[n]=np.expand_dims(mask, axis=2)

100%|██████████████████████████████████████████████████████████████████████████████| 1176/1176 [01:00<00:00, 19.56it/s]


In [147]:
for n,id_ in tqdm(enumerate(original_ids),total=len(original_ids)):
    path=original_path+"/"+id_
    img = np.load(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    W[n]=np.expand_dims(img, axis=2)

100%|██████████████████████████████████████████████████████████████████████████████| 1176/1176 [00:39<00:00, 29.93it/s]


In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [148]:
W_train, W_test, V_train, V_test = train_test_split(W, Y, test_size=0.1, random_state=42)

In [24]:
model_with_image_preprocess=tf.keras.models.load_model("lungnodule_vanilla_higher_epoch.h5",compile=False)

In [200]:
overall_dice_score = 0
for i in range(0,len(X_test)):  
    c = np.expand_dims(X_test[i], axis=0)
    y_pred=model_with_image_preprocess.predict(c)
    y_pred_thresholded = y_pred > 0.7
    d = np.squeeze(y_pred_thresholded, axis=0)
    dice_score = (2*(d*Y_test[i]).sum()) / ((d+Y_test[i]).sum() + 1e-12)
    overall_dice_score = overall_dice_score + dice_score
#     fig = plt.figure(figsize=(15, 15))
#     fig.add_subplot(1, 4, 1)
#     plt.imshow(W_test[i],cmap='gray')
#     plt.title("2D slice from CT scan")
#     fig.add_subplot(1, 4, 2)
#     plt.imshow(X_test[i],cmap='gray')
#     plt.title("2D slice after image processing")
#     fig.add_subplot(1, 4, 3)
#     plt.imshow(Y_test[i],cmap='gray')
#     plt.title("Ground truth mask of nodule")
#     fig.add_subplot(1, 4, 4)
#     plt.imshow(d,cmap='gray')
#     plt.title("Predicted mask of nodule")
    #plt.savefig("Evaluated_images/with_prepreprocess/sample"+ str(i)+" "+'dice_score'+" "+str(int(np.ceil(dice_score*100))), bbox_inches='tight')
print("Dice score on test dataset is: ", overall_dice_score)
#plt.savefig('Evaluated_images/vanilla_with_prepreprocess/sample'+ str(i), bbox_inches='tight')   

1/1 [==============================] - 0s 96ms/step
Dice score on test dataset is:  49.43799550416613
